Build a regression model.

In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [14]:
bike_poi_df = pd.read_csv('../data/joined_bikes.csv')
bike_poi_df = bike_poi_df.drop('Unnamed: 0',axis=1)
bike_poi_df = bike_poi_df[bike_poi_df['distance_meters'] < 201]
bike_poi_df.describe()

,distance_meters,popularity,rating,lat,long,bikes_available,bikes_in_use,total_bikes
count,45889.000000,15384.000000,5985.000000,45889.000000,45889.000000,45889.000000,45889.000000,45889.000000
mean,60.389331,0.696245,7.148204,52.510455,13.389849,1.847763,3.354006,5.156399
std,25.338322,0.306045,1.035183,0.025355,0.053063,2.352336,2.439783,3.252509
min,0.000000,0.000088,4.100000,52.438212,13.228195,0.000000,0.000000,2.000000
25%,41.000000,0.453704,6.400000,52.494314,13.346871,0.000000,2.000000,4.000000
50%,61.000000,0.854558,7.200000,52.510176,13.396425,1.000000,3.000000,4.000000
75%,80.000000,0.944794,7.900000,52.527044,13.428717,3.000000,4.000000,4.000000
max,200.000000,0.999940,9.500000,52.637125,13.607622,17.000000,22.000000,33.000000


In [ ]:
# I need to make a model that demonstrates a relationship between the number of bikes at a station and the charactaristics
# of the POIs there.
import seaborn as sns
sns.heatmap(bike_poi_df.corr())
# Looking at my sad heatmap again I think I'll have to 'unpack' the data in the 'category' column to be binary.
# For example I could add a columns called 'near_restaurant'/'near_cafe'/'residential'/etc where 0 is False and 1 is True.
# This would be tied to origin like total_bikes is. One issue that immediatley pops into mind with this is the 'poi density' of 
# different origins. Some have 50 POIs tied to them. Some have 5. I'll have to group by origin, total_bikes, and the categorical
# columns to void this. Essentially unjoining the two tables..

In [16]:
bike_poi_df['category'] = bike_poi_df['category'].str.lower()
# Just in case

In [17]:
bike_poi_df['category'].value_counts().head()

business and professional services    2985
clothing store                        1154
attorney / law office                 1066
hair salon                             952
café                                   865
Name: category, dtype: int64

In [ ]:
#           declare new column               do a lambda transform so we can evaluate every category in each grouped origin
bike_poi_df['near_restaurant'] = bike_poi_df.groupby('origin')['category'].transform(
    lambda x: x.str.contains('restaurant').any()
    ).map({True: 1, False: 0})
#   set the value of the new column


# Run this to get a feel of the impact this new column will have. for example take technology business at 384 occurences.
bike_poi_df.groupby('origin')['category'].transform(
    lambda x: x.str.contains('business').any()
    ).value_counts()
# if we try this chunk of code with contains('business') it's way too vague and will change nearly 40k rows.
# if we try it with ('technology') it only changes 10k rows.
# we can also do to see exactly what categories it's picking up.
bike_poi_df[bike_poi_df['category'].str.contains('technology',na=False)].category.value_counts()
# In this case it's only referencing 'technology business' I think it's a good add.

bike_poi_df['near_tech'] = bike_poi_df.groupby('origin')['category'].transform(
    lambda x: x.str.contains('technology').any()
    ).map({True: 1, False: 0})
bike_poi_df.head(1)

# I am going to keep testing and adding these binary columns. I'll just print the final database in the cell below to avoid clutter.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
bike_poi_df = pd.read_csv('../data/joined_bikes_expanded.csv') # If you reload the file run this.
bike_poi_df = bike_poi_df.drop('Unnamed: 0',axis=1)
bike_poi_df.head(1)
# I zoned out and we're at nearly 100 columns now...
# bike_poi_df.to_csv('../data/joined_bikes_expanded.csv') save before I lose my work.

,name,distance_meters,category,popularity,rating,lat,long,origin,bikes_available,bikes_in_use,...,near_language_school,near_music_school,near_nursery_school,near_driving_school,near_publisher,near_pub,near_bookstore,near_library,near_bridge,near_landmark
0,Curry Wolf,29.0,fast food restaurant,0.903641,7.8,52.503902,13.335662,"52.504157,13.335328",4,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(20,15))
sns.heatmap(bike_poi_df.corr())
# needles to say, this is too chaotic to digest. It does seem like there's more connections though so that's good.

# total bikes will be dependent.

In [3]:
# only grab columns relevant to station
stations = bike_poi_df[bike_poi_df.columns[7:].to_list()]

In [34]:
distinct_stations = stations.drop_duplicates()
distinct_stations = distinct_stations.drop(['bikes_in_use','bikes_available'],axis=1)
y = distinct_stations['total_bikes']
distinct_stations = distinct_stations.drop(['total_bikes','origin','near_landmark'],axis=1)
X_tmp = distinct_stations[distinct_stations.columns.to_list()]
col = X_tmp.columns
X = sm.add_constant(X_tmp)

In [ ]:
def elimination(X,y,col):

    while len(col)>0 :
        model=sm.OLS(y,X[col])
        result=model.fit()
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.0045):
            return result
            break
        else:
            col=col.drop(largest_pvalue.index)
            print('dropped',largest_pvalue)

result = elimination(X,y,col)

In [6]:
distinct_stations = stations.drop_duplicates()
distinct_stations = distinct_stations.drop(['bikes_in_use','bikes_available'],axis=1)
y = distinct_stations['total_bikes']
distinct_stations = distinct_stations.drop(['total_bikes','origin','near_landmark'],axis=1)
indep = distinct_stations[distinct_stations.columns.to_list()]
X = [sm.add_constant(indep[column]) for column in indep.columns] 

In [15]:
# As a challenge I took Jeremy's forward selection code from his lecture and automated it.

# y is still the dependent var.
# X this time is a list containing each column in x that has been turned into a df with a constant.
def addition(X,y):
    df_list = list()
    # List of columns that have been selected.

    best_r = -1
    r_val = 0
    # while params

    remaining_var = distinct_stations
    # filtered dataframe that does not contain selected columns.
    while(best_r < r_val):

        Models = [sm.OLS(y,x) for x in X] #list of models
        Results = [model.fit() for model in Models] #list of results
        # print(len(Results))
        Adj_Rsquared = [results.rsquared_adj for results in Results] #list of rsquared

        # Find highest r value and the column it came from.
        tmp_dict = dict()
        for i in range(len(Adj_Rsquared)):
            tmp_dict[remaining_var.columns[i]] = Adj_Rsquared[i]

        tmp_df = pd.DataFrame.from_dict(tmp_dict.items())
        r_col = tmp_df.max().values[0]
        r_val = tmp_df.max().values[1]
        # print(r_val)
        # print(r_col)

        # Add our column to the list.
        df_list.append(r_col)

        # drop the selected column from the pool.
        remaining_var = remaining_var.drop(r_col,axis=1)

        # columns that we have selected and will be brought forward.
        included_df =  distinct_stations[df_list]

        # X is now a list of dataframes of format (constant, selected columns, remaining column)
        # Each remaining column that needs to be tested will have a place in this list under the above format.
        X = [sm.add_constant(pd.merge(included_df,remaining_var[column], right_index = True, left_index = True)) for column in remaining_var.columns]
        # print(X[0]) uncomment this to better visualize it.
        best_r = r_val
        
        
    return df_list
        

selection = addition(X,y)
selection

# It was bittersweet to see this run as intended but also immediately return it's best selection..
# You can make the while loop run forever and print the r_Val and r_col to get an idea of how it's 'thinking'
# This will throw an error when there's nothing else to select so you won't have to restart your kernel.

# I think there could be application for this but one takeaway is that forward selection is not the way to go when you have a ton of
# variables to filter through.
# If I had more time I would look at making variations of this for example:
#   a function that does many 'seeded' runs of this and returns the best one. (start at random column instead of best column first.. you may as well 
#   try a start off every variable and return the best option at that point though.)
#   Run the selection that is returned from the elimation function. We already have a decent r value there and this could be a quick test to see if it could be improved
#   You could also try run the selection minus the most recently added one.
#   I think this has potential to produce decent results as the elimination is purely looking at pval so feeding it so forward selection then gives a final check to
#   see if any better r vals can be taken.

['near_website_designer']

Provide model output and an interpretation of the results. 

In [30]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            total_bikes   R-squared (uncentered):                   0.603
Model:                            OLS   Adj. R-squared (uncentered):              0.600
Method:                 Least Squares   F-statistic:                              222.4
Date:                Sun, 29 Jan 2023   Prob (F-statistic):                        0.00
Time:                        12:41:35   Log-Likelihood:                         -4823.4
No. Observations:                1771   AIC:                                      9671.
Df Residuals:                    1759   BIC:                                      9736.
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
near_restaurant          1.5946      0.187      8.528      0.000       1.228       1.961
near_prof_service        2.4438      0.165     14.780      0.000       2.119       2.768
near_bakery              0.8151      0.201      4.059      0.000       0.421       1.209
near_bar                 0.7035      0.196      3.589      0.000       0.319       1.088
near_organization        1.3876      0.222      6.246      0.000       0.952       1.823
near_park                1.1595      0.378      3.070      0.002       0.419       1.900
near_museum              1.1550      0.370      3.118      0.002       0.429       1.881
near_prof_cleaner        0.9313      0.259      3.596      0.000       0.423       1.439
near_computer_repair     0.8232      0.277      2.971      0.003       0.280       1.366
near_playground          1.0397      0.308      3.380      0.001       0.436       1.643
near_plaza               1.2112      0.282      4.296      0.000       0.658       1.764
near_bridge              1.8387      0.398      4.619      0.000       1.058       2.619
==============================================================================
Omnibus:                      714.302   Durbin-Watson:                   1.100
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3739.905
Skew:                           1.842   Prob(JB):                         0.00
Kurtosis:                       9.092   Cond. No.                         5.46
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### R-value
The r values indicate that the model is unable to explain 40% of the variance of the dependent variable.

Online it seems like the consensus is that values of .6 plus display a moderate correlation.

I think this makes sense. Take this plot from my eda showing total bikes represented by the size of the marker.

![failed to load](https://cdn.discordapp.com/attachments/1063653051602321462/1069354545005731870/image.png)

In the heart of Berlin there's a ton of these stations with 4-6 total bikes. Here and there there's stations with ~20-30 total_bikes. Due to the tight grouping, even with a relatively small radius they're bound to have very similiar poi's associated with them.

I think it's these, for lack of a better word, outliers that are greatly responsible for our r values.

I Googled 'regression model that's for outliers than OLS' and the wiki for robust regression popped up so I searched for the statsmodel application of it.

I tried it out and found the summary doesn't go quite as deep as an OLS summary. I noticed it did use some more independent variables though. I think to properly compare them you'd have to do a regression plot and so on but I'll spare you from another long tangent.

### F-test
The p-value for the f-test is very low so we know our model is a much better fit than a empty model.
### Log likelihood
We don't have another model to compare at the moment so this is more or less meaningless.

BIC and AIC is also relative so we'll ignore it.
### Coefficients
near_prof_service is one I didn't expect to have such a high coef. Let's scatter plot it to see where these are.

![failed to load](https://cdn.discordapp.com/attachments/1063653051602321462/1069364318803607674/image.png)

There's a ton of them. They're by a lot of, maybe even almost all of the lower total_bikes stations but they're also buy all of the stations with lots of total_bikes.

It's hard to tell exactly how this is effecting my model but my model seems to really like being near them.

Otherwise the first thing I noticed was bridge, park, playground and their significance. This aligns with the patterns I noticed when I first started using these maps where the most popular and biggest bike stations are by parks,nature, or bike paths of some sort. If our API had better means of detecting these locations these correlations would be even stronger.

Also I initially expected bars would have a stronger correlation to bike sharing with drunk driving and all. I'm not sure why that popped into mind considering biking in heavy traffic while drunk is a lot more dangerous than just getting a cab haha.

![failed to load](https://cdn.discordapp.com/attachments/1063653051602321462/1069388793976717392/image.png)

Here's a residual plot of the model. Looking at its worst point the jump from 4 to 26 bikes under the same criteria is pretty brutal. I do still think it makes note of valuable patterns in the data but I don't think it's going to be used as a prediction model too well.

I played around for a while fine tuning the independent categories being passed through but kept getting worse r values and about the same residual plot.

### Final Remark
The first thing I'll say is university campuses are the cash cows of the bike sharing world. Buch campus being a glowing example. By their dorms and shopping center they have high volume bike stations that are in constant very high demand, by their classes they have more bike stations also high in demand and right on campus they have a massive bike sharing station on the edge of the park so all the young people can go out and bike around the trails. Every station on that campus is used to its full potential. A lot of the students are active, a lot of the students don't have a lot of money or a car, and a lot of the students do fun things. Needless to say if a company has the oportunity to put their bikesharing stations on and around a campus they should take it over every other opportunity.

However if you want your stations to collect dust you can put small capacity stations on every block of the inner city. Do they make the city more bike accessible? Sure. Do they put a healthy return into the pocket of a private company? No. Leave it to the government. Locals who cycle likely have their own bikes, and tourists are far more likely to rent a bike and cycle around the waterfront, a beautiful park, or an old city shopping plaza designed for foot traffic than do circles around your apartment complex. Do these stations get used? yes but if you're looking for a station that can justify having a large capacity and utilize the majority of that capacity nearly every day these aren't the types of stations you want. If you set up a bunch of these stations you're also likely to have some that are prone to vandalism/theft/misuse which cuts further into your profits.

It's worth noting that 'tourist trap' places like landmarks and famous spots will naturaly be more active during tourist season. A more consistent activity and more use from locals would could from setting up stations by parks/bike paths.

In summary: campuses are your ideal first choice, nature, walkways, landmarks, foot traffic plazas come next.

# Stretch

How can you turn the regression model into a classification model?

In [57]:
# add a column called 'many_bikes' with 0 being no and 1 being yes.
binary_df = stations.drop_duplicates()
binary_df = binary_df.drop(['bikes_in_use','bikes_available'],axis=1)
# we know the majority of values in total_bikes is 4 and looking at the percentiles even at 75% total_bikes is still at 4
# For this reason I think we can consider anything above 4 to be large.
binary_df.head(1)
binary_df.describe()

,total_bikes,near_restaurant,near_tech,near_grocery,near_prof_service,near_clothing,near_consulting,near_attorney,near_hair_salon,near_nail_salon,...,near_language_school,near_music_school,near_nursery_school,near_driving_school,near_publisher,near_pub,near_bookstore,near_library,near_bridge,near_landmark
count,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,...,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.000000,1771.0
mean,5.032185,0.551666,0.163749,0.289102,0.674195,0.285714,0.155280,0.280632,0.333710,0.060418,...,0.006776,0.005647,0.003388,0.002823,0.098250,0.074534,0.120271,0.002823,0.050254,0.0
std,2.980840,0.497464,0.370152,0.453474,0.468807,0.451882,0.362273,0.449435,0.471671,0.238327,...,0.082059,0.074952,0.058124,0.053074,0.297736,0.262712,0.325370,0.053074,0.218531,0.0
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,4.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,33.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [58]:
# Make column in one pass using lambda transform with map again.
binary_df['many_bikes'] = binary_df['total_bikes'].transform(
    lambda x: x > 4).map({True: 1, False: 0})

In [52]:
binary_df[['many_bikes','total_bikes']].groupby(by='many_bikes').count()
# Everything looks good so lets make a logistic regression model now.

,total_bikes
many_bikes,
0,1455
1,316


In [ ]:
y = binary_df['many_bikes']
binary_df = binary_df.drop(['total_bikes','many_bikes','origin','near_landmark'],axis=1)
X_tmp = binary_df[binary_df.columns.to_list()]
col = X_tmp.columns
X = sm.add_constant(X_tmp)
def elimination(X,y,col):

    while len(col)>0 :
        model=sm.Logit(y,X[col])
        result=model.fit()
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.0045):
            return result
            break
        else:
            col=col.drop(largest_pvalue.index)
            print('dropped',largest_pvalue)

log_result = elimination(X,y,col)

In [54]:
log_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             many_bikes   No. Observations:                 1771
Model:                          Logit   Df Residuals:                     1766
Method:                           MLE   Df Model:                            4
Date:                Sun, 29 Jan 2023   Pseudo R-squ.:                -0.08839
Time:                        15:37:21   Log-Likelihood:                -904.02
converged:                       True   LL-Null:                       -830.61
Covariance Type:            nonrobust   LLR p-value:                     1.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
near_restaurant      -0.4756      0.111     -4.281      0.000      -0.693      -0.258
near_grocery         -0.5757      0.141     -4.090      0.000      -0.852      -0.300
near_prof_service    -0.9162      0.102     -8.999      0.000      -1.116      -0.717
near_firm            -0.5322      0.174     -3.066      0.002      -0.872      -0.192
near_accountant      -0.5649      0.185     -3.056      0.002      -0.927      -0.203
=====================================================================================
"""

so, this is a logistic regression classification model but not a good one. The negative pseudo r-squa means it performs worse than a null model. On the bright side the log-likelihood seems relatively low, compared to my last models at least. One neat thing is the coefs are all negative. This means the model would initially assume a higher total_bikes value then based on if these columns were true it would decrease that count.

In [60]:
log_result.summary()
# Here's another one with a p-value threshold at <0.05
# Different but nothing to write home about.

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             many_bikes   No. Observations:                 1771
Model:                          Logit   Df Residuals:                     1761
Method:                           MLE   Df Model:                            9
Date:                Sun, 29 Jan 2023   Pseudo R-squ.:                -0.07314
Time:                        15:45:39   Log-Likelihood:                -891.36
converged:                       True   LL-Null:                       -830.61
Covariance Type:            nonrobust   LLR p-value:                     1.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
near_restaurant              -0.3167      0.122     -2.599      0.009      -0.555      -0.078
near_grocery                 -0.4919      0.144     -3.408      0.001      -0.775      -0.209
near_prof_service            -0.8240      0.109     -7.586      0.000      -1.037      -0.611
near_hair_salon              -0.3390      0.142     -2.389      0.017      -0.617      -0.061
near_tea_and_coffee          -0.3060      0.148     -2.072      0.038      -0.595      -0.017
near_firm                    -0.5296      0.176     -3.005      0.003      -0.875      -0.184
near_accountant              -0.5633      0.186     -3.024      0.002      -0.928      -0.198
near_club                    -0.3594      0.164     -2.187      0.029      -0.681      -0.037
near_community_government     0.4931      0.198      2.486      0.013       0.104       0.882
near_language_school          1.2517      0.610      2.052      0.040       0.056       2.447
=============================================================================================
"""